In [10]:

import pandas as pd
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kendalltau
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from scipy.stats import mannwhitneyu, kruskal
import numpy as np
import math
import scipy.stats

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import des données :
df = pd.read_csv( "/content/drive/MyDrive/AI in Insurance/training.csv", sep=';')
df.head()

,PolNum,CalYear,Gender,Type,Category,Occupation,Age,Group1,Bonus,Poldur,Value,Adind,SubGroup2,Group2,Density,Exppdays,Numtppd,Numtpbi,Indtppd,Indtpbi
1,200114978,2009,Male,C,Large,Employed,25,18,90,3,15080,0,L46,L,72.012883,365,1,0,0.0,0.0
2,200114994,2009,Male,E,Large,Employed,20,11,30,2,22370,1,O38,O,39.550411,365,1,0,0.0,0.0
3,200115001,2009,Female,E,Large,Unemployed,42,11,150,0,39650,0,Q28,Q,169.529148,365,2,0,0.0,0.0
4,200115011,2009,Female,C,Medium,Housewife,21,5,0,0,12600,1,L6,L,58.894688,365,1,0,0.0,0.0
5,200115015,2009,Female,D,Large,Employed,33,12,30,10,9065,0,N4,N,109.631885,365,2,0,0.0,0.0


<h1>Data Ceaning</>

In [5]:
# Get general information on the dataset (ncol, nrow, nbr missing values, dtypes)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100021 entries, 1 to 100021
Data columns (total 20 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   PolNum      100021 non-null  int64  
 1   CalYear     100021 non-null  int64  
 2   Gender      100021 non-null  object 
 3   Type        100021 non-null  object 
 4   Category    100021 non-null  object 
 5   Occupation  100021 non-null  object 
 6   Age         100021 non-null  int64  
 7   Group1      100021 non-null  int64  
 8   Bonus       100021 non-null  int64  
 9   Poldur      100021 non-null  int64  
 10  Value       100021 non-null  int64  
 11  Adind       100021 non-null  int64  
 12  SubGroup2   100021 non-null  object 
 13  Group2      100021 non-null  object 
 14  Density     100021 non-null  float64
 15  Exppdays    100021 non-null  int64  
 16  Numtppd     100021 non-null  int64  
 17  Numtpbi     100021 non-null  int64  
 18  Indtppd     100021 non-null  float64
 19  In

In [7]:
# Check for missing values in the training dataset
print("\nMissing Values in Training Dataset:")
print(df.isnull().sum())


Missing Values in Training Dataset:
PolNum        0
CalYear       0
Gender        0
Type          0
Category      0
Occupation    0
Age           0
Group1        0
Bonus         0
Poldur        0
Value         0
Adind         0
SubGroup2     0
Group2        0
Density       0
Exppdays      0
Numtppd       0
Numtpbi       0
Indtppd       0
Indtpbi       0
dtype: int64


In [8]:
# Check for duplicates in the training dataset
print("\nDuplicates in Training Dataset:")
print(df.duplicated().sum())


Duplicates in Training Dataset:
0


**<h1>Handling Outliers**

**Define the list of numerical columns**

numerical_cols = ['Indtppd', 'Indtpbi']

**Iterate over each numerical column**

for col in numerical_cols: # Calculate the IQR Q1 = training_data[col].quantile(0.05) Q3 = training_data[col].quantile(0.95) IQR = Q3 - Q1

efine the bounds for the outliers
lower_bound = Q1 - 0.5 * IQR
upper_bound = Q3 + 0.5 * IQR

Cap the outliers
training_data[col + '_Capped'] = training_data[col].apply(lambda x: min(max(x, lower_bound), upper_bound))

In [11]:
# Handling outliers based on standard deviations

# Define the list of numerical columns
cols_with_outliers = ['Indtpbi', 'Indtppd', 'Bonus']

# Iterate over each numerical column
for col in cols_with_outliers:
    # Calculate mean, standard deviation, and threshold
    mean = np.mean(df[col])
    std = np.std(df[col])
    threshold = 3  # I chose 3 as the number of standard deviations to use as threshold

    # Find outliers
    outliers = df[col][np.abs(df[col] - mean) > threshold * std]

    # Create a new column with processed values (replacing outliers with NaN)
    processed_col_name = col + '_processed'
    df[processed_col_name] = df[col].where(~df[col].isin(outliers))

In [12]:
# Replace null values with median
df['Indtpbi_processed'].fillna(df['Indtpbi_processed'].median(), inplace=True)
df['Indtppd_processed'].fillna(df['Indtppd_processed'].median(), inplace=True)
df['Bonus_processed'].fillna(df['Bonus_processed'].median(), inplace=True)

In [13]:
# Summary statistics of the training dataset
print("\nSummary Statistics of Training Dataset:")
df.describe()


Summary Statistics of Training Dataset:


,PolNum,CalYear,Age,Group1,Bonus,Poldur,Value,Adind,Density,Exppdays,Numtppd,Numtpbi,Indtppd,Indtpbi,Indtpbi_processed,Indtppd_processed,Bonus_processed
count,1.000210e+05,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000
mean,2.002003e+08,2009.499895,41.122514,10.692625,-6.921646,5.470781,16454.675268,0.512142,117.159270,327.588007,0.147449,0.046790,106.135007,222.762829,53.431082,50.582517,-10.143570
std,6.217239e+04,0.500002,14.299349,4.687286,48.633165,4.591194,10506.742732,0.499855,79.500907,73.564636,0.436917,0.219546,444.949188,1859.422836,409.267436,193.913012,44.038319
min,2.001149e+08,2009.000000,18.000000,1.000000,-50.000000,0.000000,1000.000000,0.000000,14.377142,91.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-50.000000
25%,2.001399e+08,2009.000000,30.000000,7.000000,-40.000000,1.000000,8380.000000,0.000000,50.625783,340.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-40.000000
50%,2.001649e+08,2009.000000,40.000000,11.000000,-30.000000,4.000000,14610.000000,1.000000,94.364623,365.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-30.000000
75%,2.002608e+08,2010.000000,51.000000,14.000000,10.000000,9.000000,22575.000000,1.000000,174.644525,365.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000
max,2.002858e+08,2010.000000,75.000000,20.000000,150.000000,15.000000,49995.000000,1.000000,297.385170,365.000000,7.000000,3.000000,12878.369910,69068.026292,5796.313276,1440.666933,130.000000


In [14]:
print("Valeurs uniques de la colonne Numtppd:", df['Numtppd'].unique())
print("Valeurs uniques de la colonne Numtpbi:", df['Numtpbi'].unique())
print("Valeurs uniques de la colonne Indtppd:", df['Indtppd'].unique())
print("Valeurs uniques de la colonne Indtpbi:", df['Indtpbi'].unique())
print("Valeurs uniques de la colonne Bonus:", df['Bonus'].unique())

Valeurs uniques de la colonne Numtppd: [1 2 0 3 4 5 6 7]
Valeurs uniques de la colonne Numtpbi: [0 1 2 3]
Valeurs uniques de la colonne Indtppd: [   0.          740.30229036  207.32365675 ...  546.40798246  822.92046036
 5800.0189068 ]
Valeurs uniques de la colonne Indtpbi: [0.00000000e+00 3.84004453e+03 2.79371442e+03 ... 3.52825919e+00
 1.05603349e+03 1.65076419e+01]
Valeurs uniques de la colonne Bonus: [ 90  30 150   0  40  80 -20 -40 120 -30  20  50 -50 -10 100  10 110 140
  70  60 130]


In [15]:
# Select only numeric columns
numeric_cols = df.select_dtypes(include=['number'])  # Select only numeric columns

# Calculer le 90e percentile pour chaque colonne numérique
percentile_90 = numeric_cols.quantile(0.9)

# Compter le nombre de valeurs au-dessus du 90e percentile dans chaque colonne
count_above_90 = (numeric_cols > percentile_90).sum()

# Calculer le pourcentage des valeurs au-dessus du 90e percentile dans chaque colonne
percentage_above_90 = (count_above_90 / len(numeric_cols)) * 100

# Afficher les pourcentages
print("Pourcentage des valeurs au-dessus du 90e percentile dans chaque colonne :")
print(percentage_above_90)

Pourcentage des valeurs au-dessus du 90e percentile dans chaque colonne :
PolNum               9.999900
CalYear              0.000000
Age                  9.530998
Group1               7.824357
Bonus                8.996111
Poldur               7.149499
Value                9.991902
Adind                0.000000
Density              9.701963
Exppdays             0.000000
Numtppd              1.918597
Numtpbi              4.505054
Indtppd              9.999900
Indtpbi              4.505054
Indtpbi_processed    3.283311
Indtppd_processed    9.999900
Bonus_processed      8.764160
dtype: float64


**Second method : replacing by the median value :**

**Select only numeric columns**

numeric_cols = df.select_dtypes(include=['number']) # Select only numeric columns

**Outlier detection using IQR**

Q1 = numeric_cols.quantile(0.25) Q3 = numeric_cols.quantile(0.75) IQR = Q3 - Q1 lower_bound = Q1 - 0.5 * IQR upper_bound = Q3 + 0.5 * IQR

**Identify outliers**

outliers = (numeric_cols < lower_bound) | (numeric_cols > upper_bound)

**Handling outliers: Replace outliers with the median value**

training_data_no_outliers = numeric_cols.mask(outliers, numeric_cols.median(), axis=0)

**Fill remaining NaN values with median column-wise**

training_data_no_outliers = training_data_no_outliers.fillna(training_data_no_outliers.median())

**Summary statistics of the training dataset**

print("\nSummary Statistics of Training Dataset:") training_data_no_outliers.describe()

In [18]:
#Second method : replacing by the median value :

# Select only numeric columns
numeric_cols = df.select_dtypes(include=['number'])  # Select only numeric columns

# Outlier detection using IQR
Q1 = numeric_cols.quantile(0.25)
Q3 = numeric_cols.quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 0.5 * IQR
upper_bound = Q3 + 0.5 * IQR

# Identify outliers
outliers = (numeric_cols < lower_bound) | (numeric_cols > upper_bound)

# Handling outliers: Replace outliers with the median value
df_no_outliers = numeric_cols.mask(outliers, numeric_cols.median(), axis=0)

# Fill remaining NaN values with median column-wise
df_no_outliers = df_no_outliers.fillna(df_no_outliers.median())

# Afficher les valeurs uniques de chaque colonne
print("Valeurs uniques de la colonne Numtppd:", df['Numtppd'].unique())
print("Valeurs uniques de la colonne Numtpbi:", df['Numtpbi'].unique())
print("Valeurs uniques de la colonne Indtppd:", df['Indtppd'].unique())
print("Valeurs uniques de la colonne Indtpbi:", df['Indtpbi'].unique())

# Summary statistics of the training dataset
print("\nSummary Statistics of Training Dataset:")
df_no_outliers.describe()

Valeurs uniques de la colonne Numtppd: [1 2 0 3 4 5 6 7]
Valeurs uniques de la colonne Numtpbi: [0 1 2 3]
Valeurs uniques de la colonne Indtppd: [   0.          740.30229036  207.32365675 ...  546.40798246  822.92046036
 5800.0189068 ]
Valeurs uniques de la colonne Indtpbi: [0.00000000e+00 3.84004453e+03 2.79371442e+03 ... 3.52825919e+00
 1.05603349e+03 1.65076419e+01]

Summary Statistics of Training Dataset:


,PolNum,CalYear,Age,Group1,Bonus,Poldur,Value,Adind,Density,Exppdays,Numtppd,Numtpbi,Indtppd,Indtpbi,Indtpbi_processed,Indtppd_processed,Bonus_processed
count,1.000210e+05,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.000000,100021.0,100021.0,100021.0,100021.0,100021.0,100021.0,100021.000000
mean,2.002003e+08,2009.499895,38.632917,10.777337,-26.512732,4.720279,13994.279751,0.512142,95.442840,364.289149,0.0,0.0,0.0,0.0,0.0,0.0,-26.512732
std,6.217239e+04,0.500002,10.229251,3.314991,21.589932,3.850623,7070.331768,0.499855,58.300440,4.171336,0.0,0.0,0.0,0.0,0.0,0.0,21.589932
min,2.001149e+08,2009.000000,20.000000,4.000000,-50.000000,0.000000,1285.000000,0.000000,14.377142,328.000000,0.0,0.0,0.0,0.0,0.0,0.0,-50.000000
25%,2.001399e+08,2009.000000,31.000000,9.000000,-40.000000,1.000000,8475.000000,0.000000,50.625783,365.000000,0.0,0.0,0.0,0.0,0.0,0.0,-40.000000
50%,2.001649e+08,2009.000000,38.000000,11.000000,-30.000000,4.000000,13100.000000,1.000000,77.688233,365.000000,0.0,0.0,0.0,0.0,0.0,0.0,-30.000000
75%,2.002608e+08,2010.000000,46.000000,13.000000,-10.000000,8.000000,19160.000000,1.000000,137.887334,365.000000,0.0,0.0,0.0,0.0,0.0,0.0,-10.000000
max,2.002858e+08,2010.000000,61.000000,17.000000,30.000000,13.000000,29670.000000,1.000000,236.399384,365.000000,0.0,0.0,0.0,0.0,0.0,0.0,30.000000


Cependant nous nous sommes rendus compte que la notion d'outliers en fonction de notre contexte ne faisait pas sens. En effet, dans la partie exploration des données il nous a été donné de voir la répartition de nos variables. Par exemple pour la fréquence de sinistres (qui semble présenter des outlier à priori), cela serait du au fait que la plupart ees assurés n'on jamais eu à déclarer de sinistres beaucoup ont eu à en déclarer 1 ou 2 et plus rarement entre 3 et 7. Ces valeurs pour les catégories 3 à 7, déséquilibrées, sont donc considérées comme outliers alors qu'il faut les garder dans notre contexte